In [2]:
import numpy as np
from numpy import fft
import gmmhmm
import MFCC
import glob
import pickle
from scipy.misc import imread
from matplotlib import pyplot as plt
from sklearn.feature_extraction.image import extract_patches_2d
from scipy import linalg as la
%matplotlib inline

MFCC.py:35: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  M[r,c+1] += 2 * (1 - pm[c])
MFCC.py:38: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  M[r,c+1] += 2 * pm[c]


For the KDEF Database, we want series one, female and male subjects, at a straight on angle, expressing the emotions of afraid, angry, disgusted, happy, neutral, and sad.

These consist of images of the form "A(F|M)(01-35)(AF|AN|DI|HA|NE|SA)S.jpg"

In [11]:
def get_sampledicts():
    emotions = ['AF','AN','DI','HA','NE','SA']
    full_emotions = {'AF':'Fear','AN':'Anger','DI':'Disgust',\
                     'HA':'Happiness','NE':'Neutral','SA':'Sadness'}
    
    sampledict = {}
    for emotion in emotions:
        sampledict[emotion] = []
        for sex in ['M','F']:
            for k in xrange(1,3):
                if len(str(k)) == 1:
                    k = '0'+str(k)
                else:
                    k = str(k)
                fname = 'KDEF\KDEF\A'+sex+k+'\A'+sex+k+emotion+'S.jpg'
                # Image size is now 380x280
                img = imread(fname,flatten=True)[:-2:2,:-2:2]
                
                num_patches = 239
                patches = extract_patches_2d(img,(10,10),max_patches=num_patches)
                
                
                print fft.fft2(patches[0,:,:],(10,1)).shape
                
                """
                features = np.zeros((num_patches,10))
                for i in xrange(num_patches):
                    U, s, Vt = la.svd(patches[i,:,:])
                    features[i,:] = s
                
                
                U, s, Vt = la.svd(img)
                print U.shape
                print s.shape
                s = s.reshape(1,-1)
                print s.shape
                
                img = img.astype(np.int16)
                img = img.ravel()
                
                extracted = MFCC.extract(img)
                print extracted.shape
                sampledict[emotion].append(extracted)
                #plt.imshow(img.reshape((380,280)),cmap='Greys_r')
                #plt.show()
                """
                
    
    return emotions, sampledict

# Problem 3
def train(names, samples):
    best_models = []
    for name in names:
        print name
        best = -np.inf
        best_model = None
        for i in xrange(10):
            print "Test number ", i
            startprob, transmat = initialize(5)
            model = gmmhmm.GMMHMM(n_components=5, n_mix=3, transmat=transmat, startprob=startprob, cvtype='diag')
            # these values for covars_prior and var should work well for this problem
            model.covars_prior = 0.01
            model.fit(samples[name][:20], init_params='mc', var=0.1)
            print "Training on ", name, "result: ", model.logprob, 'at time ', time.asctime(time.localtime())
            if model.logprob > best:
                best = model.logprob
                best_model = model
                #f = open(model_name(name),'w')
                #pickle.dump(model,f)
                #print "New best prob: ", best
                #f.close()
        best_models.append(best_model)
    #print best_models
    return best_models

def test(names, samples, models):
    results = []
    for name in names:
        for i in xrange(20,30):
            best = -np.inf
            best_model = -1
            for j in xrange(len(models)):
                score = models[j].score(i)
                if score > best:
                    best = score
                    best_model = j
            results.append(best_model)
            
    results = np.array(results)
    return results

In [46]:
data = np.loadtxt('mini_emotion_results.txt')
arrs = np.split(data,[50,100,150,200,250])
results = 0
for i in xrange(len(arrs)):
    result = (50 - np.count_nonzero(arrs[i]-i))/50.
    results += result
    print result
print 'Overall: ',results/6.
    
data1 = np.loadtxt('emotion_results1.txt')
arrs = np.split(data1,6)
results = 0
for i in xrange(6):
    result = (20 - np.count_nonzero(arrs[i] - i))/20.
    results += result
    print result
print 'Overall: ', results/6.

0.22
0.46
0.04
0.26
0.36
0.1
Overall:  0.24
0.2
0.4
0.35
0.15
0.1
0.2
Overall:  0.233333333333


In [1]:
# Patch method: 239 patches was pretty abysmal, only 20% accuracy. Trying enough
# patches to get the whole image didn't really help much. Notice that disgust 
# has a pretty high thing for both things

In [69]:
import pandas as pd
import seaborn
from pandas.tools.plotting import table
data = pd.DataFrame()

emotion_pairs = ['AF/AN','AF/DI','AF/HA','AF/NE','AF/SA','AN/DI','AN/HA','AN/NE',
                'AN/SA','DI/HA','DI/NE','DI/SA','HA/NE','HA/SA','NE/SA','Overall']
column_titles = ['C','SH','SL','SHC','FHC','FLC','FL','Average']

pair_results = [[.55,.58,.6,.53,.5,.58,.48],
               [.58,.48,.5,.48,.5,.58,.45],
               [.58,.5,.6,.75,.6,.65,.5],
               [.55,.53,.58,.53,.63,.48,.55],
               [.5,.5,.5,.48,.53,.53,.53],
               [.55,.6,.58,.63,.63,.65,.53],
               [.7,.63,.48,.6,.58,.5,.45],
               [.68,.6,.68,.55,.7,.6,.63],
               [.63,.53,.55,.53,.6,.5,.53],
               [.45,.58,.6,.58,.5,.55,.6],
               [.5,.5,.58,.6,.65,.55,.48],
               [.55,.5,.53,.5,.53,.6,.48],
               [.63,.53,.55,.55,.53,.53,.5],
               [.58,.5,.55,.6,.53,.48,.58],
               [.63,.6,.63,.63,.55,.5,.55]]
pair_results = np.array(pair_results)

method_avg = np.mean(pair_results,axis=0)

pair_results = np.vstack((pair_results,method_avg))
pair_avg = np.mean(pair_results,axis=1)

pair_results = np.hstack((pair_results,pair_avg.reshape((16,1))))

for i in xrange(8):
    column_dict = {}
    for j in xrange(16):
        column_dict[emotion_pairs[j]] = pair_results[j,i]
    data[column_titles[i]] = pd.Series(column_dict)
"""
ax = plt.subplot(111, frame_on=False) # no visible frame
ax.xaxis.set_visible(False)  # hide the x axis
ax.yaxis.set_visible(False)  # hide the y axis

table(ax, data)  # where df is your data frame
plt.savefig('emotion_pair_table.png')

fig = plt.figure()
ax2 = seaborn.heatmap(data,annot=True,xticklabels=True,yticklabels=True,linewidths=.5,cmap='YlGnBu',cbar=False)
ax = fig.add_subplot(ax2)
fig.savefig('seaborn_emotion_pairs.png')
"""

"\nax = plt.subplot(111, frame_on=False) # no visible frame\nax.xaxis.set_visible(False)  # hide the x axis\nax.yaxis.set_visible(False)  # hide the y axis\n\ntable(ax, data)  # where df is your data frame\nplt.savefig('emotion_pair_table.png')\n\nfig = plt.figure()\nax2 = seaborn.heatmap(data,annot=True,xticklabels=True,yticklabels=True,linewidths=.5,cmap='YlGnBu',cbar=False)\nax = fig.add_subplot(ax2)\nfig.savefig('seaborn_emotion_pairs.png')\n"

In [70]:
data1 = np.loadtxt('emotion_results1.txt')
data1 = data1.astype('int64')
emotions = ['AF','AN','DI','HA','NE','SA']
arrs = np.split(data1,6)
print np.array(arrs).shape
results = 0
df = pd.DataFrame()
for i in xrange(6):
    c_dict = {}
    counters = np.bincount(arrs[i])
    for j in xrange(6):
        c_dict[emotions[j]] = counters[j]
    df[emotions[i]] = pd.Series(c_dict)
    
df = df.transpose()

#fig = plt.figure()
#ax3 = df.plot(kind='bar')
#plt.savefig('example_emotion_guesses.png')

(6L, 20L)
